In [2]:
using DataFrames
using Dates
using Plots
using Statistics,RData  #upload data of R format 
using CategoricalArrays # categorical data
using DataFrames
using CSV
using Tables
using Lathe
using GLM
using Lasso
using StatsPlots
using CovarianceMatrices
using MLBase

In [30]:
rdata_read = load("wage2015_subsample_inference.RData")
data = rdata_read["data"]
names(data)
println("Number of Rows : ", size(data)[1],"\n","Number of Columns : ", size(data)[2],) #rows and columns

Number of Rows : 5150
Number of Columns : 20


In [43]:
data[!,:occ] = convert.(String,data[!,:occ])
data[!,:occ2] = convert.(String,data[!,:occ2])
data[!,:ind] = convert.(String,data[!,:ind])
data[!,:ind2] = convert.(String,data[!,:ind2])

1376-element Vector{String}:
 "4"
 "14"
 "9"
 "19"
 "18"
 "18"
 "4"
 "4"
 "9"
 "20"
 "14"
 "14"
 "6"
 ⋮
 "8"
 "8"
 "16"
 "19"
 "20"
 "20"
 "10"
 "22"
 "19"
 "9"
 "21"
 "11"

In [44]:
[eltype(col) for col = eachcol(data)]

20-element Vector{DataType}:
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 String
 String
 String
 String

In [45]:
data = filter(row ->(row.shs == 1 || row.hsg == 1), data)

,wage,lwage,sex,shs,hsg,scl,clg,ad,mw
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,11.0577,2.40313,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,19.2308,2.95651,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,19.2308,2.95651,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,12.0,2.48491,1.0,0.0,1.0,0.0,0.0,0.0,0.0
5,17.3077,2.85115,1.0,0.0,1.0,0.0,0.0,0.0,0.0
6,12.0192,2.48651,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,27.8846,3.32808,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,21.6,3.07269,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,19.2308,2.95651,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [57]:
first(data,10)

,wage,lwage,sex,shs,hsg,scl,clg,ad,mw
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,11.0577,2.40313,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,19.2308,2.95651,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,19.2308,2.95651,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,12.0,2.48491,1.0,0.0,1.0,0.0,0.0,0.0,0.0
5,17.3077,2.85115,1.0,0.0,1.0,0.0,0.0,0.0,0.0
6,12.0192,2.48651,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,27.8846,3.32808,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,21.6,3.07269,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,19.2308,2.95651,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [59]:
#LASSO
# using hmd.jl
include("hdmjl.jl")

In [60]:
#First we test the BASIC MODEL
#definig Y D & W
Y = data[!,"lwage"]
Y = DataFrame([Y], [:Y])
D = data[!,"sex"]
D = DataFrame([D],[:D])
W = DataFrame(select(data, Not(["lwage", "sex", "exp2","exp3","exp4","occ","ind","ad","wage","ne","scl","clg"])))

,shs,hsg,mw,so,we,exp1,occ2,ind2
,Float64,Float64,Float64,Float64,Float64,Float64,String,String
1,0.0,1.0,0.0,0.0,0.0,18.0,19,4
2,0.0,1.0,0.0,0.0,0.0,42.0,17,14
3,0.0,1.0,0.0,0.0,0.0,37.0,17,9
4,0.0,1.0,0.0,0.0,0.0,31.0,13,19
5,0.0,1.0,0.0,0.0,0.0,7.0,13,18
6,0.0,1.0,0.0,0.0,0.0,30.0,14,18
7,0.0,1.0,0.0,0.0,0.0,16.0,19,4
8,0.0,1.0,0.0,0.0,0.0,27.0,19,4
9,0.0,1.0,0.0,0.0,0.0,6.0,20,9


In [61]:
#First equation Y
res_Y_0 = rlasso_arg( W, Y, nothing, true, true, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )

rlasso_arg(1376×8 DataFrame. Omitted printing of 1 columns
│ Row  │ shs     │ hsg     │ mw      │ so      │ we      │ exp1    │ occ2   │
│      │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │ String │
├──────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼────────┤
│ 1    │ 0.0     │ 1.0     │ 0.0     │ 0.0     │ 0.0     │ 18.0    │ 19     │
│ 2    │ 0.0     │ 1.0     │ 0.0     │ 0.0     │ 0.0     │ 42.0    │ 17     │
│ 3    │ 0.0     │ 1.0     │ 0.0     │ 0.0     │ 0.0     │ 37.0    │ 17     │
│ 4    │ 0.0     │ 1.0     │ 0.0     │ 0.0     │ 0.0     │ 31.0    │ 13     │
│ 5    │ 0.0     │ 1.0     │ 0.0     │ 0.0     │ 0.0     │ 7.0     │ 13     │
│ 6    │ 0.0     │ 1.0     │ 0.0     │ 0.0     │ 0.0     │ 30.0    │ 14     │
│ 7    │ 0.0     │ 1.0     │ 0.0     │ 0.0     │ 0.0     │ 16.0    │ 19     │
│ 8    │ 0.0     │ 1.0     │ 0.0     │ 0.0     │ 0.0     │ 27.0    │ 19     │
│ 9    │ 0.0     │ 1.0     │ 0.0     │ 0.0     │ 0.0     │ 6.0     │ 20     │
│ 10 

In [62]:
#Put parameters inside lasso reg
res_Y = rlasso(res_Y_0)

LoadError: MethodError: no method matching /(::String, ::Int64)
[0mClosest candidates are:
[0m  /(::Any, [91m::ChainRulesCore.AbstractThunk[39m) at C:\Users\Alvaro\.julia\packages\ChainRulesCore\RbX5a\src\tangent_types\thunks.jl:33
[0m  /([91m::Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}[39m, ::Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}) at C:\Users\Alvaro\AppData\Local\Programs\Julia-1.7.2\share\julia\base\int.jl:93
[0m  /([91m::StridedArray{P}[39m, ::Real) where P<:Period at C:\Users\Alvaro\AppData\Local\Programs\Julia-1.7.2\share\julia\stdlib\v1.7\Dates\src\deprecated.jl:44
[0m  ...

In [49]:
#calling residuals row
res_Y = rlasso(res_Y_0)["residuals"]

LoadError: MethodError: no method matching /(::String, ::Int64)
[0mClosest candidates are:
[0m  /(::Any, [91m::ChainRulesCore.AbstractThunk[39m) at C:\Users\Alvaro\.julia\packages\ChainRulesCore\RbX5a\src\tangent_types\thunks.jl:33
[0m  /([91m::Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}[39m, ::Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}) at C:\Users\Alvaro\AppData\Local\Programs\Julia-1.7.2\share\julia\base\int.jl:93
[0m  /([91m::StridedArray{P}[39m, ::Real) where P<:Period at C:\Users\Alvaro\AppData\Local\Programs\Julia-1.7.2\share\julia\stdlib\v1.7\Dates\src\deprecated.jl:44
[0m  ...

In [54]:
#second equation D (same steps)
res_D_0 = rlasso_arg( W, D, nothing, true, true, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )
#calling residuals
res_D = rlasso(res_D_0)["residuals"]
# convert the vector into matrix 
res_D = reshape(res_D, length(res_D), 1)

LoadError: MethodError: no method matching /(::String, ::Int64)
[0mClosest candidates are:
[0m  /(::Any, [91m::ChainRulesCore.AbstractThunk[39m) at C:\Users\Alvaro\.julia\packages\ChainRulesCore\RbX5a\src\tangent_types\thunks.jl:33
[0m  /([91m::Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}[39m, ::Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}) at C:\Users\Alvaro\AppData\Local\Programs\Julia-1.7.2\share\julia\base\int.jl:93
[0m  /([91m::StridedArray{P}[39m, ::Real) where P<:Period at C:\Users\Alvaro\AppData\Local\Programs\Julia-1.7.2\share\julia\stdlib\v1.7\Dates\src\deprecated.jl:44
[0m  ...

In [55]:
lm(res_D, res_Y)

LoadError: UndefVarError: res_D not defined

In [ ]:
#Now we test de FLEXIBLE MODEL
flex_y = @formula(lwage ~ (exp1+exp2+exp3+exp4+shs+hsg+occ2+ind2+mw+so+we)^2)

In [ ]:
res_f_0 = rlasso_arg( flex_y, nothing, true, true, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )
#calling residuals row
res_f = rlasso(res_f_0)["residuals"]
# convert the vector into matrix 
res_f = reshape(res_f, length(res_f), 1)

In [ ]:
#Now we test the regresion D Y
flex_d = @formula(sex ~ (exp1+exp2+exp3+exp4+shs+hsg+occ2+ind2+mw+so+we)^2)

In [ ]:
res_f_D_0 = rlasso_arg(flex_d, nothing, true, true, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )

In [ ]:
#finally the regressor errors
lm(res_f_D, res_f)